In [24]:
import numpy as np
import math
from interval import imath
from interval import fpu
from interval import interval

Напишем алгоритм, который будет находить экстремум функции вдоль заданного направления.
Алгоритм будет основываться на методе "Золотого сечения" так как является наиболее эффективным по стравнению с основными методами одномерной оптимизации.

In [25]:
def argmin(f, index, a, b, p, e):  #f(function), i(index of direction), a(left border), b(right border), p(current point), e(error)
    phi = (1 + math.sqrt(5)) / 2  #constant of golden ratio
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = p.copy()  #current point
    p_2 = p.copy()  #current point
    p_1[index] = x_1
    p_2[index] = x_2
    f_1 = f(p_1)  #value in 1-st point
    f_2 = f(p_2)  #value in 2-nd point
    while b - a > e:  #termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_2 = f_1
            f_1 = f(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_1 = f_2
            f_2 = f(p_2)
    mid = p_1
    for i in range(len(mid)):
        mid[i] = (p_1[i] + p_2[i]) / 2
    return mid  #point of extremum with error e

Функция вычисляющая евклидово расстояние между 2 точками

In [26]:
def r(p_1, p_2):
    r_ = 0
    for i in range(len(p_1)):
        r_ += (p_1[i] - p_2[i]) ** 2
    return math.sqrt(r_)

Функция принимающая функцию, множество определения и начальную точку.
Она раелизует метод наискорейшего спуска, последовательно ищет минимум вдоль направлений параллельных координатным осям и останавливается когда улучшение точности становится малым.

In [35]:
def fast_search(f, D, p, e, e_n, method):  #f(function), D(set), p(start point), e(error)
    dimension = len(p)

    while True:
        p_0 = p
        for i in range(dimension):
            p = method(f, i, D[i][0], D[i][1], p, e)
        if r(p_0, p) < e_n:
            break
    return p

Рассмотрим пример работы функции на примере:

In [36]:
def F(x):
    return (x[0] ** 2) / 2 - (x[1] ** 2) / 4 + 3


p = fast_search(F, [[-10, 10], [-10, 10]], [1, 1], 0.01, 0.001, argmin)  #point of minimum
y = F(p)  #minimum value
print(p)
print(y)

[-0.002800335820726037, -9.995468961462151]
-21.977346018947962


Попробуем теперь применить тут алгоритм Moore-Skelboe для поиска минимума вдоль направления:

In [38]:
def Moore_Skelboe(f, index, a, b, p, e):
    interval_d = [None] * len(p)
    for i in range(len(p)):
        interval_d[i] = interval[p[i], p[i]]
    interval_d[index] = interval[a, b]

    interval_e = f(interval_d)
    list_of_intervals = [[interval_d, interval_e]]
    U = fpu.max(interval_e)[1]
    w = len(interval_e)
    best_interval = list_of_intervals[0]
    while w > e:
        list_of_intervals.pop(0)
        mid = (fpu.max(best_interval[0][index])[0] + fpu.max(best_interval[0][index])[1]) / 2
        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        interval_1[index] = interval[fpu.max(best_interval[0][index])[0], mid]
        interval_1e = f(interval_1)
        interval_2[index] = interval[mid, fpu.max(best_interval[0][index])[1]]
        interval_2e = f(interval_2)
        U = min(U, fpu.max(interval_1e)[1])
        U = min(U, fpu.max(interval_2e)[1])
        list_of_intervals.append([interval_1, interval_1e])
        list_of_intervals.append([interval_2, interval_2e])
        for el in list_of_intervals:
            if U < fpu.max(el[1])[0]:
                list_of_intervals.remove(el)
        list_of_intervals.sort(key=lambda item: fpu.max(item[1])[0])
        best_interval = list_of_intervals[0]
        w = fpu.max(best_interval[0][index])[1] - fpu.max(best_interval[0][index])[0]

    point = [None] * len(p)
    for i in range(len(p)):
        point[i] = fpu.max(best_interval[0][i])[0]
    return point

In [39]:
p = fast_search(F, [[-10, 10], [-10, 10]], [1, 1], 0.001, 0.001, Moore_Skelboe)
y = F(p)
print(p)
print(y)

[-0.0006103515625, -10.0]
-21.999999813735485
